# Google Earth Engine Component

## Initialize

In [1]:
#Import required libraries
import ee
import geemap
import math

In [2]:
#Initialize Google Earth Engine
#ee.Authenticate() just needed the 1st time
ee.Initialize()

In [3]:
# Check if geemap is working as intended - plot the leaflet map
Map = geemap.Map()

## Load Feature Collection - Shapefiles

In [4]:
#Data loads

#loads feature collection data from Google Earth Engine - We can also upload other feature collections
counties = ee.FeatureCollection("TIGER/2018/Counties")

#filter LA County
la_county = counties.filter(ee.Filter.eq('NAME', 'Los Angeles'))

In [5]:
#Income Data

la_county_income = ee.FeatureCollection("projects/california-lawn-detection/assets/lacountyincome-final")

#Map.addLayer(la_county_income,{}, 'LA County Income')

In [6]:
la_county_income2 = ee.FeatureCollection("projects/california-lawn-detection/assets/lacountyincome_update")
la_county_income = la_county_income2.select(ee.List(['Name', 'Descriptio', 'Ranking']), ee.List(['Name', 'Median_Income', 'Ranking']))
Map.addLayer(la_county_income,{}, 'LA County Income')

In [7]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Load NAIP Imagery

In [8]:
#loads NAIP 2020 and 2010 dataset and selects bands Red, Green, Blue, Near Infrared (R,G,B,N)

# rgbn imagery from 2010 to 2020
image_2020_rgbn = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2020-01-01', '2020-12-31')
    .select(['R', 'G', 'B', 'N'])
)

image_2018_rgbn = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2018-01-01', '2018-12-31')
    .select(['R', 'G', 'B', 'N'])
)

image_2016_rgbn = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2016-01-01', '2016-12-31')
    .select(['R', 'G', 'B', 'N'])
)

image_2014_rgbn = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2014-01-01', '2014-12-31')
    .select(['R', 'G', 'B', 'N'])
)

image_2012_rgbn = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2012-01-01', '2012-12-31')
    .select(['R', 'G', 'B', 'N'])
)

image_2010_rgbn = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2010-01-01', '2010-12-31')
    .select(['R', 'G', 'B', 'N'])
)

# nrg imagery from 2010 to 2020
image_2020_nrg = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2020-01-01', '2020-12-31')
    .select(['N','R','G'])
)

image_2018_nrg = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2018-01-01', '2018-12-31')
    .select(['N','R','G'])
)

image_2016_nrg = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2016-01-01', '2016-12-31')
    .select(['N','R','G'])
)

image_2014_nrg = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2014-01-01', '2014-12-31')
    .select(['N','R','G'])
)

image_2012_nrg = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2012-01-01', '2012-12-31')
    .select(['N','R','G'])
)

image_2010_nrg = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate('2010-01-01', '2010-12-31')
    .select(['N','R','G'])
)

#Calculate median so it can be clipped - turns image collection into an image
NAIP_2020_rgbn = image_2020_rgbn.median()
NAIP_2018_rgbn = image_2018_rgbn.median()
NAIP_2016_rgbn = image_2016_rgbn.median()
NAIP_2014_rgbn = image_2014_rgbn.median()
NAIP_2012_rgbn = image_2012_rgbn.median()
NAIP_2010_rgbn = image_2010_rgbn.median()

NAIP_2020_nrg = image_2020_nrg.median()
NAIP_2018_nrg = image_2018_nrg.median()
NAIP_2016_nrg = image_2016_nrg.median()
NAIP_2014_nrg = image_2014_nrg.median()
NAIP_2012_nrg = image_2012_nrg.median()
NAIP_2010_nrg = image_2010_nrg.median()

#Clip images to desired polygon in this case la_county

NAIP_2020_rgbn_lacounty = NAIP_2020_rgbn.clip(la_county)
NAIP_2020_nrg_lacounty = NAIP_2020_nrg.clip(la_county)
NAIP_2018_rgbn_lacounty = NAIP_2018_rgbn.clip(la_county)
NAIP_2018_nrg_lacounty = NAIP_2018_nrg.clip(la_county)
NAIP_2016_rgbn_lacounty = NAIP_2016_rgbn.clip(la_county)
NAIP_2016_nrg_lacounty = NAIP_2016_nrg.clip(la_county)
NAIP_2014_rgbn_lacounty = NAIP_2014_rgbn.clip(la_county)
NAIP_2014_nrg_lacounty = NAIP_2014_nrg.clip(la_county)
NAIP_2012_rgbn_lacounty = NAIP_2012_rgbn.clip(la_county)
NAIP_2012_nrg_lacounty = NAIP_2012_nrg.clip(la_county)
NAIP_2010_rgbn_lacounty = NAIP_2010_rgbn.clip(la_county)
NAIP_2010_nrg_lacounty = NAIP_2010_nrg.clip(la_county)


In [9]:
#Calculate NDVI for each image
# ndvi_2020 = NAIP_2020_rgbn_lacounty.normalizedDifference(['N', 'R']).rename('NAIP_2020_NDVI')
# ndvi_2018 = NAIP_2018_rgbn_lacounty.normalizedDifference(['N', 'R']).rename('NAIP_2018_NDVI')
# ndvi_2016 = NAIP_2016_rgbn_lacounty.normalizedDifference(['N', 'R']).rename('NAIP_2016_NDVI')
# ndvi_2014 = NAIP_2014_rgbn_lacounty.normalizedDifference(['N', 'R']).rename('NAIP_2014_NDVI')
# ndvi_2012 = NAIP_2012_rgbn_lacounty.normalizedDifference(['N', 'R']).rename('NAIP_2012_NDVI')
# ndvi_2010 = NAIP_2010_rgbn_lacounty.normalizedDifference(['N', 'R']).rename('NAIP_2010_NDVI')

In [10]:
#Add it as a layer
#Map.addLayer(ndvi_2020, {min : -1, max : 1, 'palette' : ['blue', 'white', 'green']}, 'NAIP_2020_NDVI')

# 2018
# Map.addLayer(ndvi_2018, {min : -1, max : 1, 'palette' : ['blue', 'white', 'green']}, 'NAIP_2018_NDVI')

# 2016
# Map.addLayer(ndvi_2016, {min : -1, max : 1, 'palette' : ['blue', 'white', 'green']}, 'NAIP_2016_NDVI')

# 2014
# Map.addLayer(ndvi_2014, {min : -1, max : 1, 'palette' : ['blue', 'white', 'green']}, 'NAIP_2014_NDVI')

# 2012
# Map.addLayer(ndvi_2012, {min : -1, max : 1, 'palette' : ['blue', 'white', 'green']}, 'NAIP_2012_NDVI')

# 2010
# Map.addLayer(ndvi_2010, {min : -1, max : 1, 'palette' : ['blue', 'white', 'green']}, 'NAIP_2010_NDVI')

In [11]:
#Add layer
#Map.addLayer(NAIP_2020_rgbn_lacounty,{}, 'NAIP_2020_rgbn')
Map.addLayer(NAIP_2020_nrg_lacounty,{}, 'NAIP_2020_nrg')

#2018
#Map.addLayer(NAIP_2018_rgbn_lacounty,{}, 'NAIP_2018_rgbn')
Map.addLayer(NAIP_2018_nrg_lacounty,{}, 'NAIP_2018_nrg')

#2016
#Map.addLayer(NAIP_2016_rgbn_lacounty,{}, 'NAIP_2016_rgbn')
Map.addLayer(NAIP_2016_nrg_lacounty,{}, 'NAIP_2016_nrg')

#2014
#Map.addLayer(NAIP_2014_rgbn_lacounty,{}, 'NAIP_2014_rgbn')
Map.addLayer(NAIP_2014_nrg_lacounty,{}, 'NAIP_2014_nrg')

#2012
#Map.addLayer(NAIP_2012_rgbn_lacounty,{}, 'NAIP_2012_rgbn')
Map.addLayer(NAIP_2012_nrg_lacounty,{}, 'NAIP_2012_nrg')

#2010
#Map.addLayer(NAIP_2010_rgbn_lacounty,{}, 'NAIP_2010_rgbn')
Map.addLayer(NAIP_2010_nrg_lacounty,{}, 'NAIP_2010_nrg')

In [12]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Texture

In [13]:
def add_NDVI(image):
    image = image.addBands(image.normalizedDifference(['N','R']).rename('NDVI'))
    return image

In [14]:
NAIP_2020_rgbn_ndvi_lacounty = add_NDVI(NAIP_2020_rgbn_lacounty)
NAIP_2018_rgbn_ndvi_lacounty = add_NDVI(NAIP_2018_rgbn_lacounty)
NAIP_2016_rgbn_ndvi_lacounty = add_NDVI(NAIP_2016_rgbn_lacounty)
NAIP_2014_rgbn_ndvi_lacounty = add_NDVI(NAIP_2014_rgbn_lacounty)
NAIP_2012_rgbn_ndvi_lacounty = add_NDVI(NAIP_2012_rgbn_lacounty)
NAIP_2010_rgbn_ndvi_lacounty = add_NDVI(NAIP_2010_rgbn_lacounty)

In [15]:
def neighborhood_bands(image, band):
    i_8_bit = image.select(band).toUint8()
    square = ee.Kernel.square(**{'radius': 4})
    entropy = i_8_bit.entropy(square)
    glcm = i_8_bit.glcmTexture(**{'size': 4})
    contrast = glcm.select(str(band)+'_contrast')
    
    # Create a list of weights for a 9x9 kernel.
    list = [1, 1, 1, 1, 1, 1, 1, 1, 1]
    # The center of the kernel is zero.
    centerList = [1, 1, 1, 1, 0, 1, 1, 1, 1]
    # Assemble a list of lists: the 9x9 kernel weights as a 2-D matrix.
    lists = [list, list, list, list, centerList, list, list, list, list]
    # Create the kernel from the weights.
    # Non-zero weights represent the spatial neighborhood.
    kernel = ee.Kernel.fixed(9, 9, lists, -4, -4, False)
    neighs = i_8_bit.neighborhoodToBands(kernel)
    gearys = i_8_bit.subtract(neighs).pow(2).reduce(ee.Reducer.sum()).divide(math.pow(9, 2))
    image = image.addBands(entropy.rename(str(band)+'_Entropy')).addBands(contrast.rename(str(band)+'_Contrast')).addBands(gearys.rename(str(band)+'_Gearys'))   
    return image


In [16]:
bands = ['N']
for band in bands:
    NAIP_2020_rgbn_ndvi_lacounty = neighborhood_bands(NAIP_2020_rgbn_ndvi_lacounty,band)
    NAIP_2018_rgbn_ndvi_lacounty = neighborhood_bands(NAIP_2018_rgbn_ndvi_lacounty,band)
    NAIP_2016_rgbn_ndvi_lacounty = neighborhood_bands(NAIP_2016_rgbn_ndvi_lacounty,band)
    NAIP_2014_rgbn_ndvi_lacounty = neighborhood_bands(NAIP_2014_rgbn_ndvi_lacounty,band)
    NAIP_2012_rgbn_ndvi_lacounty = neighborhood_bands(NAIP_2012_rgbn_ndvi_lacounty,band)
    NAIP_2010_rgbn_ndvi_lacounty = neighborhood_bands(NAIP_2010_rgbn_ndvi_lacounty,band)

### Machine Learning Section

In [44]:
# Get predictions

bands = ['R', 'G', 'B', 'N', 'NDVI', 'N_Entropy', 'N_Contrast','N_Gearys']

la_image_2020 = NAIP_2020_rgbn_ndvi_lacounty.select(bands)
la_image_2018 = NAIP_2018_rgbn_ndvi_lacounty.select(bands)
la_image_2016 = NAIP_2016_rgbn_ndvi_lacounty.select(bands)
la_image_2014 = NAIP_2014_rgbn_ndvi_lacounty.select(bands)
la_image_2012 = NAIP_2012_rgbn_ndvi_lacounty.select(bands)
la_image_2010 = NAIP_2010_rgbn_ndvi_lacounty.select(bands)

In [45]:
# Define classes
CLASSES = ['water',
           'vegetation_trees',
           'vegetation_grass',
           'turf',
           'impervious',
           'soil']

# Change classes to include lakes
CLASSES[0] = 'pools'
CLASSES_P1 = CLASSES + ['lakes']

In [46]:
# Point to the specific TF model to be used for inference
PROJECT = 'w210-351617'
MODEL_NAME = 'CNN_Nbands_sep_cls_wlake_model'
VERSION_NAME = 'v0'
input_dim = [12,12]

# Point to the model hosted on AI Platform.
new_tf_model = ee.Model.fromAiPlatformPredictor(
    projectName=PROJECT,
    modelName=MODEL_NAME,
    version=VERSION_NAME,
    # Can be anything, but don't make it too big.
    inputTileSize=input_dim,
    # Note the names here need to match what was specified in the
    # output dictionary passed to the EEifier originally
    outputBands={'output': {
        'type': ee.PixelType.float(),
        'dimensions': 1
      }
    },
)

In [47]:
# Get the predictions
predictions_2020 = new_tf_model.predictImage(la_image_2020.float().toArray())
probabilities_2020 = predictions_2020.arrayFlatten([CLASSES_P1])
label_2020 = predictions_2020.arrayArgmax().arrayGet([0]).rename('label')

In [48]:
predictions_2018 = new_tf_model.predictImage(la_image_2018.float().toArray())
probabilities_2018 = predictions_2018.arrayFlatten([CLASSES_P1])
label_2018 = predictions_2018.arrayArgmax().arrayGet([0]).rename('label')

In [49]:
predictions_2016 = new_tf_model.predictImage(la_image_2016.float().toArray())
probabilities_2016 = predictions_2016.arrayFlatten([CLASSES_P1])
label_2016 = predictions_2016.arrayArgmax().arrayGet([0]).rename('label')

In [50]:
predictions_2014 = new_tf_model.predictImage(la_image_2014.float().toArray())
probabilities_2014 = predictions_2014.arrayFlatten([CLASSES_P1])
label_2014 = predictions_2014.arrayArgmax().arrayGet([0]).rename('label')

In [51]:
predictions_2012 = new_tf_model.predictImage(la_image_2012.float().toArray())
probabilities_2012 = predictions_2012.arrayFlatten([CLASSES_P1])
label_2012 = predictions_2012.arrayArgmax().arrayGet([0]).rename('label')

In [52]:
predictions_2010 = new_tf_model.predictImage(la_image_2010.float().toArray())
probabilities_2010 = predictions_2010.arrayFlatten([CLASSES_P1])
label_2010 = predictions_2010.arrayArgmax().arrayGet([0]).rename('label')

In [53]:
# Point to the model hosted on AI Platform.  If you specified a region other
# than the default (us-central1) at model creation, specify it here.
# Point to the specific TF model to be used for inference
PROJECT = 'w210-351617'
MODEL_NAME = 'CNN_Nbands_model'
VERSION_NAME = 'v0'
input_dim = [12,12]

# Point to the model hosted on AI Platform.
old_tf_model = ee.Model.fromAiPlatformPredictor(
    projectName=PROJECT,
    modelName=MODEL_NAME,
    version=VERSION_NAME,
    # Can be anything, but don't make it too big.
    inputTileSize=input_dim,
    # Note the names here need to match what was specified in the
    # output dictionary passed to the EEifier originally
    outputBands={'output': {
        'type': ee.PixelType.float(),
        'dimensions': 1
      }
    },
)


In [54]:
old_predictions_2020 = old_tf_model.predictImage(la_image_2020.float().toArray())
old_probabilities_2020 = old_predictions_2020.arrayFlatten([CLASSES])
old_label_2020 = old_predictions_2020.arrayArgmax().arrayGet([0]).rename('label')
old_probabilities_lakes_2020 = old_probabilities_2020.addBands(probabilities_2020, ['lakes'])


In [55]:
old_predictions_2018 = old_tf_model.predictImage(la_image_2018.float().toArray())
old_probabilities_2018 = old_predictions_2018.arrayFlatten([CLASSES])
old_label_2018 = old_predictions_2018.arrayArgmax().arrayGet([0]).rename('label')
old_probabilities_lakes_2018 = old_probabilities_2018.addBands(probabilities_2018, ['lakes'])


In [56]:
old_predictions_2016 = old_tf_model.predictImage(la_image_2016.float().toArray())
old_probabilities_2016 = old_predictions_2016.arrayFlatten([CLASSES])
old_label_2016 = old_predictions_2016.arrayArgmax().arrayGet([0]).rename('label')
old_probabilities_lakes_2016 = old_probabilities_2016.addBands(probabilities_2016, ['lakes'])


In [57]:
old_predictions_2014 = old_tf_model.predictImage(la_image_2014.float().toArray())
old_probabilities_2014 = old_predictions_2014.arrayFlatten([CLASSES])
old_label_2014 = old_predictions_2014.arrayArgmax().arrayGet([0]).rename('label')
old_probabilities_lakes_2014 = old_probabilities_2014.addBands(probabilities_2014, ['lakes'])


In [58]:
old_predictions_2012 = old_tf_model.predictImage(la_image_2012.float().toArray())
old_probabilities_2012 = old_predictions_2012.arrayFlatten([CLASSES])
old_label_2012 = old_predictions_2012.arrayArgmax().arrayGet([0]).rename('label')
old_probabilities_lakes_2012 = old_probabilities_2012.addBands(probabilities_2012, ['lakes'])


In [59]:
old_predictions_2010 = old_tf_model.predictImage(la_image_2010.float().toArray())
old_probabilities_2010 = old_predictions_2010.arrayFlatten([CLASSES])
old_label_2010 = old_predictions_2010.arrayArgmax().arrayGet([0]).rename('label')
old_probabilities_lakes_2010 = old_probabilities_2010.addBands(probabilities_2010, ['lakes'])


In [60]:
# Combine the predictions by getting the max probability class for each pixel
combined_probabilities_2020 = probabilities_2020.max(old_probabilities_lakes_2020)
combined_classified_image_2020 = combined_probabilities_2020.toArray().arrayArgmax().arrayGet([0]).rename('label')

In [61]:
combined_probabilities_2018 = probabilities_2018.max(old_probabilities_lakes_2018)
combined_classified_image_2018 = combined_probabilities_2018.toArray().arrayArgmax().arrayGet([0]).rename('label')

In [62]:
combined_probabilities_2016 = probabilities_2016.max(old_probabilities_lakes_2016)
combined_classified_image_2016 = combined_probabilities_2016.toArray().arrayArgmax().arrayGet([0]).rename('label')

In [63]:
combined_probabilities_2014 = probabilities_2014.max(old_probabilities_lakes_2014)
combined_classified_image_2014 = combined_probabilities_2014.toArray().arrayArgmax().arrayGet([0]).rename('label')

In [64]:
combined_probabilities_2012 = probabilities_2012.max(old_probabilities_lakes_2012)
combined_classified_image_2012 = combined_probabilities_2012.toArray().arrayArgmax().arrayGet([0]).rename('label')

In [65]:
combined_probabilities_2010 = probabilities_2010.max(old_probabilities_lakes_2010)
combined_classified_image_2010 = combined_probabilities_2010.toArray().arrayArgmax().arrayGet([0]).rename('label')

In [66]:
# Add layers to map
legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911', '#191970']
Map.addLayer(label_2020, {'min': 0, 'max': 6, 'palette': legend_colors}, 'new_classification')

legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911']
Map.addLayer(old_label_2020, {'min': 0, 'max': 5, 'palette': legend_colors}, 'old_classification')


In [71]:
legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911', '#191970']
Map.addLayer(label_2012, {'min': 0, 'max': 6, 'palette': legend_colors}, '12_new_classification')

legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911']
Map.addLayer(old_label_2012, {'min': 0, 'max': 5, 'palette': legend_colors}, '12_old_classification')

In [69]:
legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911', '#191970']
Map.addLayer(combined_classified_image_2020, {'min': 0, 'max': 6, 'palette': legend_colors}, 'combined_classification_20')
Map.addLayer(combined_classified_image_2018, {'min': 0, 'max': 6, 'palette': legend_colors}, 'combined_classification_18')
Map.addLayer(combined_classified_image_2016, {'min': 0, 'max': 6, 'palette': legend_colors}, 'combined_classification_16')
Map.addLayer(combined_classified_image_2014, {'min': 0, 'max': 6, 'palette': legend_colors}, 'combined_classification_14')
Map.addLayer(combined_classified_image_2012, {'min': 0, 'max': 6, 'palette': legend_colors}, 'combined_classification_12')
Map.addLayer(combined_classified_image_2010, {'min': 0, 'max': 6, 'palette': legend_colors}, 'combined_classification_10')

In [73]:
added_probabilities_12 = probabilities_2012.add(old_probabilities_lakes_2012)
added_classified_image_12 = added_probabilities_12.toArray().arrayArgmax().arrayGet([0]).rename('label')

Map.addLayer(added_classified_image_12, {'min': 0, 'max': 6, 'palette': legend_colors}, '12_added_classification')

In [74]:
Map

Map(bottom=13393952.333343506, center=[34.09143957343236, -118.27522516250612], controls=(WidgetControl(option…

In [20]:
# Get predictions
predictions_2020 = final_model.predictImage(la_image_2020.float().toArray())
probabilities_2020 = predictions_2020.arrayFlatten([training_classes_list_text])
label_2020 = predictions_2020.arrayArgmax().arrayGet([0]).rename('label')

In [21]:
predictions_2018 = final_model.predictImage(la_image_2018.float().toArray())
probabilities_2018 = predictions_2018.arrayFlatten([training_classes_list_text])
label_2018 = predictions_2018.arrayArgmax().arrayGet([0]).rename('label')

In [22]:
predictions_2016 = final_model.predictImage(la_image_2016.float().toArray())
probabilities_2016 = predictions_2016.arrayFlatten([training_classes_list_text])
label_2016 = predictions_2016.arrayArgmax().arrayGet([0]).rename('label')

In [23]:
predictions_2014 = final_model.predictImage(la_image_2014.float().toArray())
probabilities_2014 = predictions_2014.arrayFlatten([training_classes_list_text])
label_2014 = predictions_2014.arrayArgmax().arrayGet([0]).rename('label')

In [24]:
predictions_2012 = final_model.predictImage(la_image_2012.float().toArray())
probabilities_2012 = predictions_2012.arrayFlatten([training_classes_list_text])
label_2012 = predictions_2012.arrayArgmax().arrayGet([0]).rename('label')

In [25]:
predictions_2010 = final_model.predictImage(la_image_2010.float().toArray())
probabilities_2010 = predictions_2010.arrayFlatten([training_classes_list_text])
label_2010 = predictions_2010.arrayArgmax().arrayGet([0]).rename('label')

In [26]:
legend_keys = ['water', 'vegetation_trees', 'vegetation_grass', 'turf','impervious','soil']
legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911']

Map.addLayer(label_2020,{'min': 0, 'max': 5, 'palette': legend_colors},"LA_2020")
Map.addLayer(label_2018,{'min': 0, 'max': 5, 'palette': legend_colors},"LA_2018")
Map.addLayer(label_2016,{'min': 0, 'max': 5, 'palette': legend_colors},"LA_2016")
Map.addLayer(label_2014,{'min': 0, 'max': 5, 'palette': legend_colors},"LA_2014")
Map.addLayer(label_2012,{'min': 0, 'max': 5, 'palette': legend_colors},"LA_2012")
Map.addLayer(label_2010,{'min': 0, 'max': 5, 'palette': legend_colors},"LA_2010")

In [28]:
Map.addLayer(NAIP_2018_rgbn_lacounty,{}, 'NAIP_2020_rgbn')
Map.addLayer(NAIP_2010_rgbn_lacounty,{}, 'NAIP_2010_rgbn')

In [27]:
Map

Map(bottom=754.0, center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [29]:
la_parcel_shape_filtered = ee.FeatureCollection("projects/california-lawn-detection/assets/LA_County_Parcels_Shape")\
                             .filter(ee.Filter.eq('UseType', 'Residential'))
    
la_parcel_res = la_parcel_shape_filtered.select(ee.List(['AIN', 'SitusCity','SitusZIP','SitusFullA']), 
                                                ee.List(['AIN', 'City','ZipCode','FullAddress']))

In [30]:
label_2020_res = label_2020.clip(la_parcel_res)
label_2018_res = label_2018.clip(la_parcel_res)
label_2016_res = label_2016.clip(la_parcel_res)
label_2014_res = label_2014.clip(la_parcel_res)
label_2012_res = label_2012.clip(la_parcel_res)
label_2010_res = label_2010.clip(la_parcel_res)

In [31]:
Map_2 = geemap.Map()

In [32]:
Map_2

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [33]:
vegetation_2020 = label_2020_res.expression("b('label') == 1 || b('label') == 2")
vegetation_2018 = label_2018_res.expression("b('label') == 1 || b('label') == 2")
vegetation_2016 = label_2016_res.expression("b('label') == 1 || b('label') == 2")
vegetation_2014 = label_2014_res.expression("b('label') == 1 || b('label') == 2")
vegetation_2012 = label_2012_res.expression("b('label') == 1 || b('label') == 2")
vegetation_2010 = label_2010_res.expression("b('label') == 1 || b('label') == 2")

In [34]:
delta1820 = vegetation_2018.subtract(vegetation_2020)
delta1618 = vegetation_2016.subtract(vegetation_2018)
delta1416 = vegetation_2014.subtract(vegetation_2016)
delta1214 = vegetation_2012.subtract(vegetation_2014)
delta1012 = vegetation_2010.subtract(vegetation_2012)
delta1020 = vegetation_2010.subtract(vegetation_2020)

In [35]:
Map_2.addLayer(NAIP_2018_nrg_lacounty,{}, 'NAIP_2018_nrg')
Map_2.addLayer(NAIP_2020_nrg_lacounty,{}, 'NAIP_2020_nrg')
Map_2.addLayer(NAIP_2010_nrg_lacounty,{}, 'NAIP_2010_nrg')

In [36]:
Map_2.addLayer(delta1820,{'min': -1, 'max': 1,  'palette': ['#097407','#FFFFFF','#A1A8AF']},"18-20 Change")

In [38]:
Map_2.addLayer(delta1820,{'min': -1, 'max': 1,  'palette': ['#097407','#FFFFFF','#A1A8AF']},"18-20 Change")
Map_2.addLayer(delta1618,{'min': -1, 'max': 1,  'palette': ['#097407','#FFFFFF','#A1A8AF']},"16-18 Change")
Map_2.addLayer(delta1416,{'min': -1, 'max': 1,  'palette': ['#097407','#FFFFFF','#A1A8AF']},"14-16 Change")
Map_2.addLayer(delta1214,{'min': -1, 'max': 1,  'palette': ['#097407','#FFFFFF','#A1A8AF']},"12-14 Change")
Map_2.addLayer(delta1012,{'min': -1, 'max': 1,  'palette': ['#097407','#FFFFFF','#A1A8AF']},"10-12 Change")
Map_2.addLayer(delta1020,{'min': -1, 'max': 1,  'palette': ['#097407','#FFFFFF','#A1A8AF']},"10-20 Change")

In [37]:
conversions1820_masked = delta1820.expression("b('label') == 1").selfMask()
conversions1618_masked = delta1618.expression("b('label') == 1").selfMask()
conversions1416_masked = delta1416.expression("b('label') == 1").selfMask()
conversions1214_masked = delta1214.expression("b('label') == 1").selfMask()
conversions1012_masked = delta1012.expression("b('label') == 1").selfMask()
conversions1020_masked = delta1020.expression("b('label') == 1").selfMask()

In [40]:
Map_2.addLayer(conversions1820_masked,{'palette':['#008080']},"18-20 Conversions Masked")

In [52]:
Map_2.addLayer(conversions1618_masked,{'palette':['#008080']},"16-18 Conversions Masked")
Map_2.addLayer(conversions1416_masked,{'palette':['#008080']},"14-16 Conversions Masked")
Map_2.addLayer(conversions1214_masked,{'palette':['#008080']},"12-14 Conversions Masked")
Map_2.addLayer(conversions1012_masked,{'palette':['#008080']},"10-12 Conversions Masked")

In [38]:
Map_2.addLayer(conversions1020_masked,{'palette':['#36048F']},"10-20 Conversions Masked")

In [40]:
Map_2.addLayer(la_county_income,{}, 'LA County Income')

In [39]:
Map_2

Map(bottom=754.0, center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [76]:
#import zipcode shapes so we can clip by zipcodes

la_county_income_zipcode2 = ee.FeatureCollection("projects/california-lawn-detection/assets/income_zipcode2019")
la_county_income_zipcode = la_county_income_zipcode2.select(ee.List(['zipcode', '2019zipcod','shape_area']), ee.List(['ZipCode', 'Median_Income','Area_sqft']))

In [81]:
polygon = la_county_income_zipcode.filter(ee.Filter.eq('ZipCode', '90703'))

In [82]:
polygon.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'version': 1655181821530462,
 'id': 'projects/california-lawn-detection/assets/income_zipcode2019',
 'properties': {'system:asset_size': 2545157},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-118.05474054584516, 33.850605694043175],
      [-118.05453536451552, 33.85073949500485],
      [-118.05041069576191, 33.85353537219483],
      [-118.05041069576191, 33.85356211715199],
      [-118.04960808252969, 33.85356655182243],
      [-118.04909980359321, 33.85357097734918],
      [-118.04872962553485, 33.85357547742632],
      [-118.0483595360295, 33.85357995407097],
      [-118.04824805344866, 33.85357990318768],
      [-118.04627270980006, 33.853602218212096],
      [-118.04627270980006, 33.853718141947205],
      [-118.04627270980006, 33.853762770195324],
      [-118.04627717630095, 33.853896495174105],
      [-118.04628164280166, 33.854128417445445],
      [-118.04628610930232, 33.85450739953751

In [83]:
Map_3.addLayer(polygon,{}, 'Polygon')

In [84]:
Map_3

Map(bottom=419255.33337402344, center=[33.97658211143439, -118.09238471784074], controls=(WidgetControl(option…

In [ ]:
polygon.getInfo().get('features')[i].get('geometry').get('NAME'))